In [39]:
import os
import numpy as np
import pickle
import gzip
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, random_split

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [40]:
def load_data():
    # Load the training and test data from specified paths
    train_path = '/content/flatland_train.data'
    test_path = '/content/flatland_test.data'

    X_train, y_train = pickle.load(gzip.open(train_path, 'rb'))
    X_test, y_test = pickle.load(gzip.open(test_path, 'rb'))

    # Assign placeholder labels (0) to y_test if needed
    y_test = np.zeros(len(X_test), dtype=int)


    y_train = np.where(y_train != 0, y_train - 2, y_train)  # Convert labels to 0, 1, 2, 3
    X_train = torch.tensor(X_train, dtype=torch.float32) / 255.0  # Normalize pixel values
    X_test = torch.tensor(X_test, dtype=torch.float32) / 255.0
    y_train = torch.tensor(y_train, dtype=torch.long)

    return X_train, y_train, X_test, y_test


In [41]:
def apply_sobel_filter(X):
    # Define Sobel filter kernels for x and y direction
    sobel_x = torch.tensor([[-1., 0., 1.],
                             [-2., 0., 2.],
                             [-1., 0., 1.]], device=X.device).unsqueeze(0).unsqueeze(0)

    sobel_y = torch.tensor([[-1., -2., -1.],
                             [0., 0., 0.],
                             [1., 2., 1.]], device=X.device).unsqueeze(0).unsqueeze(0)

    # Apply padding to the input images
    X_padded = F.pad(X, (1, 1, 1, 1), mode='constant', value=0)  # Padding (left, right, top, bottom)

    # Prepare a list to hold the edge images
    edge_images = []

    # Iterate over the batch of images
    for img in tqdm(X_padded, desc="Applying Sobel Filter"):
        # Apply Sobel filters to each individual image
        edge_x = F.conv2d(img.unsqueeze(0).unsqueeze(0), sobel_x)  # Convolve with sobel_x
        edge_y = F.conv2d(img.unsqueeze(0).unsqueeze(0), sobel_y)  # Convolve with sobel_y

        # Calculate the magnitude of the gradient
        edge_magnitude = torch.sqrt(edge_x ** 2 + edge_y ** 2)

        # Normalize to [0, 1] for visibility
        edge_magnitude_normalized = edge_magnitude / edge_magnitude.max()

        # Store the result and remove the extra dimension if necessary
        edge_images.append(edge_magnitude_normalized.squeeze(1))

    # Convert the list of edge images back to a tensor
    return torch.stack(edge_images)


In [42]:
def augment_data(X, y):
    # Flip images
    X_flipped_lr = torch.flip(X, dims=[2])  # Horizontal flip (left-right)
    X_flipped_ud = torch.flip(X, dims=[1])  # Vertical flip (up-down)
    X_flipped_diag = torch.flip(X_flipped_lr, dims=[1])  # Diagonal flip

    # Concatenate original and flipped images
    X_augmented = torch.cat((X, X_flipped_lr, X_flipped_ud, X_flipped_diag), dim=0)

    # Repeat labels to match augmented data size
    y_augmented = torch.cat((y, y, y, y), dim=0)  # Repeat labels 4 times

    return X_augmented, y_augmented


In [43]:
def prepare_dataloaders(X_train, y_train, batch_size=32, val_size=0.1):
    # Define augmentation transforms for on-the-fly augmentation
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20)
    ])

    # Split data into train and validation sets
    train_size = int((1 - val_size) * len(X_train))
    val_size = len(X_train) - train_size
    train_data, val_data = random_split(TensorDataset(X_train, y_train), [train_size, val_size])

    # Augmented DataLoader for training with transform
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader

In [44]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [45]:
import torch.optim as optim

def train_model(net, train_loader, val_loader, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adadelta(net.parameters(), weight_decay=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.01)

    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        print(f"Starting Epoch {epoch + 1}")

        # Training loop
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        net.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                val_loss += criterion(outputs, labels).item()

        # Scheduler step and average losses
        scheduler.step()
        print(f"Epoch {epoch + 1} - Training Loss: {running_loss / len(train_loader):.4f}, Validation Loss: {val_loss / len(val_loader):.4f}")


In [46]:
import numpy as np

def generate_predictions(net, X_test, batch_size=64):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).view(-1, 1, 50, 50).to(device)
    net.eval()
    predictions = []

    # Process test data in batches
    with torch.no_grad():
        for i in range(0, len(X_test_tensor), batch_size):
            batch = X_test_tensor[i:i + batch_size]
            outputs = net(batch)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())  # Store predictions as a list

    # Adjust predictions and format as a single string
    predictions = np.where(np.array(predictions) != 0, np.array(predictions) + 2, np.array(predictions))
    result_text = ''.join(map(str, predictions))
    return result_text


In [47]:
# Load and process the dataset
print("Loading data...")
X_train, y_train, X_test, y_test = load_data()

# Apply Sobel filter to both X_train and X_test
print("Applying Sobel filter to training and test data...")
X_train = apply_sobel_filter(X_train)
X_test = apply_sobel_filter(X_test)

# Augment training set
print("Augmenting training set...")
X_train, y_train = augment_data(X_train, y_train)

# Prepare DataLoaders for training and validation
print("Preparing DataLoaders...")
train_loader, val_loader = prepare_dataloaders(X_train, y_train)

# Initialize the model, train, and save the best model
print("Initializing the model and training...")
net = CNNModel()
train_model(net, train_loader, val_loader)

# Generate predictions on X_test
predictions = generate_predictions(net, X_test)

# Convert predictions to a single long text format and print
result_text = ''.join(map(str, predictions))
print(result_text)


Loading data...
Applying Sobel filter to training and test data...


Applying Sobel Filter: 100%|██████████| 10000/10000 [00:00<00:00, 10317.99it/s]


Augmenting training set...
Preparing DataLoaders...
Initializing the model and training...
Starting Epoch 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1 - Training Loss: 0.6771, Validation Loss: 0.2444
Starting Epoch 2
Epoch 2 - Training Loss: 0.2786, Validation Loss: 0.2715
Starting Epoch 3
Epoch 3 - Training Loss: 0.2351, Validation Loss: 0.2365
Starting Epoch 4
Epoch 4 - Training Loss: 0.2094, Validation Loss: 0.1679
Starting Epoch 5
Epoch 5 - Training Loss: 0.1918, Validation Loss: 0.1967
Starting Epoch 6
Epoch 6 - Training Loss: 0.1850, Validation Loss: 0.1798
Starting Epoch 7
Epoch 7 - Training Loss: 0.1821, Validation Loss: 0.1540
Starting Epoch 8
Epoch 8 - Training Loss: 0.1782, Validation Loss: 0.4438
Starting Epoch 9
Epoch 9 - Training Loss: 0.1746, Validation Loss: 0.8135
Starting Epoch 10
Epoch 10 - Training Loss: 0.1686, Validation Loss: 0.1511


<ipython-input-46-bc15bcfea631>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).view(-1, 1, 50, 50).to(device)


6545533343400053340545465645540463050355560333004363354356664064553303655503043350530654500355355463530403365345345040035360650004655530445455543654345046545535635636546303536656365650360305665463536335363336336465353306053566453360450054500063446060345543400453533456446546354503345300344553655053543306444055363443403030605300054555506334560544330054354500443044350645463346060306564305553645045356533336636330660644345564354536053644433653304055544504505345553050664300303444544504030306344006554333400506636544330334035643434343650403330363543435603556463440550543403430450565505303434536554356435035004453066464556036403330463455363406563043554443345350663053566650564533336350334356344353343060333043443433504546433334036630054450544503543600603355633656300553303033036463444450546453433660605604345456536403330035505464066564430553033365340544530533543635653364030035430440356534344063556033036433354550635036440504640065344340360630645656404363635646435404000565355305364603035300453665355604